# SKAL! model inference

## Input variables

In [ ]:
import os

os.environ['TF_GPU_ALLOCATOR'] = "cuda_malloc_async"

subdirs = os.listdir(os.getcwd())
# little workaround. with a properlyb built package this should be useless
if "skal" not in subdirs:
    os.chdir("..")

In [ ]:
config_path = "config/bigan/nanotwice.yaml"
weights_dir = "models/bigan/nanotwice"
test_img_path = "datasets/nanotwice/Anomalous/images/ITIA1101.tif"

## Loading detector

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

tf.keras.mixed_precision.set_global_policy('mixed_float16')

from skal.models.bigan import detector, loader
from skal.utils import utils
from skal.experiment.config import Config
from skal.visualization import plotters
from skal.models import model_choices
from skal.anomaly_map import ops

utils.set_gpu()
exp_params = utils.load_yaml_file(config_path)

config = Config(**exp_params)

loader = model_choices.LoaderFactory.get_loader(config.model['name'])
detector = loader.load_detector(config.detector, seed=config.seed, weights_dir=weights_dir)

## Loading Test Image

In [ ]:

image = Image.open("datasets/nanotwice/Anomalous/images/ITIA1101.tif")
plt.imshow(np.array(image), cmap='gray')
plt.axis('off')

In [ ]:
image = np.expand_dims(image, 0)
image = np.expand_dims(image, -1)

## Detector in action

In [ ]:
anomaly_scores = detector.score_anomalies(image)
reconstructed_image = detector.model.reconstruct_images(image)

## Visualizing Anomaly Scores

In [ ]:
anomaly_img = ops.anomaly_map_to_img(np.array(anomaly_scores))
plt.imshow(np.array(anomaly_scores[0]), vmin=0, vmax=1.0)
plt.axis('off')

## Visualizing Reconstruction

In [ ]:
plt.imshow(reconstructed_image[0], cmap='gray')
plt.axis('off')
plt.show()